In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Data transformation

In [2]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

train_data = datasets.ImageFolder('dataset/Train', transform=transform)
test_data = datasets.ImageFolder('dataset/Test', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

num_classes = len(train_data.classes)
print("Classes:", train_data.classes)

Classes: ['1Hundrednote', '2Hundrednote', '2Thousandnote', '5Hundrednote', 'Fiftynote', 'Tennote', 'Twentynote']


CNN

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class CurrencyCNN(nn.Module):
    def __init__(self, num_classes):
        super(CurrencyCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.fc1 = nn.Linear(128 * 16 * 16, 128)  # depends on image size
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

model = CurrencyCNN(num_classes)


Training

In [4]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 25
for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss/len(train_loader):.4f} | Accuracy: {100*correct/total:.2f}%")


Epoch 1/25 | Loss: 1.9821 | Accuracy: 13.73%
Epoch 2/25 | Loss: 1.8405 | Accuracy: 21.57%
Epoch 3/25 | Loss: 1.5399 | Accuracy: 34.64%
Epoch 4/25 | Loss: 1.5188 | Accuracy: 33.33%
Epoch 5/25 | Loss: 1.4214 | Accuracy: 41.83%
Epoch 6/25 | Loss: 1.3238 | Accuracy: 52.29%
Epoch 7/25 | Loss: 1.1450 | Accuracy: 50.33%
Epoch 8/25 | Loss: 1.0943 | Accuracy: 61.44%
Epoch 9/25 | Loss: 1.0170 | Accuracy: 55.56%
Epoch 10/25 | Loss: 0.9938 | Accuracy: 66.01%
Epoch 11/25 | Loss: 0.9132 | Accuracy: 68.63%
Epoch 12/25 | Loss: 0.8543 | Accuracy: 64.05%
Epoch 13/25 | Loss: 0.8078 | Accuracy: 69.28%
Epoch 14/25 | Loss: 0.6748 | Accuracy: 75.82%
Epoch 15/25 | Loss: 0.6459 | Accuracy: 74.51%
Epoch 16/25 | Loss: 0.6759 | Accuracy: 72.55%
Epoch 17/25 | Loss: 0.5026 | Accuracy: 82.35%
Epoch 18/25 | Loss: 0.5354 | Accuracy: 81.70%
Epoch 19/25 | Loss: 0.4734 | Accuracy: 83.66%
Epoch 20/25 | Loss: 0.4471 | Accuracy: 83.66%
Epoch 21/25 | Loss: 0.3456 | Accuracy: 88.89%
Epoch 22/25 | Loss: 0.2991 | Accuracy: 90.8

Testing

In [5]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100*correct/total:.2f}%")


Test Accuracy: 80.95%


Final Predict

In [6]:
from PIL import Image

def predict_image(img_path, model, transform):
    model.eval()
    image = Image.open(img_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    
    return train_data.classes[predicted.item()]

print(predict_image("1.jpg", model, transform))


2Thousandnote


In [8]:
torch.save(model, 'currency_cnn_model.pth')
